In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/test.csv /content

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/antecedents.tsv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/consequents.tsv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/simplified.tsv /content

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/outputs.antecedents/submit_results.csv /content/antecedent_results.csv
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/outputs.consequents/submit_results.csv /content/consequent_results.csv
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/outputs.simplified/submit_results.csv /content/simplified_results.csv

In [0]:
import pandas as pd

prefix = '/content/'

test_df = pd.read_csv(prefix + 'test.csv', header=None)
test_df=test_df.drop(index=0)

test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1]
})

# display 
test_df.head()

In [0]:
pip install pytorch_pretrained_bert

In [0]:
from pytorch_pretrained_bert import BertTokenizer,BertForMaskedLM
import torch
import pandas as pd
import math

bertMaskedLM = BertForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [0]:
def get_score(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    predictions=bertMaskedLM(tensor_input)
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(predictions.squeeze(),tensor_input.squeeze()).data 
    return math.exp(loss)

In [0]:
#get_score("If a few people get sick at a salad bar | wouldn't have happened")
get_score("in more careful people's shoes | wouldn't have happened")

In [0]:
import pandas as pd

prefix = '/content/'

a_df = pd.read_csv(prefix + 'antecedents.tsv', sep='\t', delimiter=None, header=None)
a_df = pd.DataFrame({
    'id': a_df[0],
    'labels':a_df[1],
    'text': a_df[2],
    'start':a_df[3],
    'end': a_df[4]
})
c_df = pd.read_csv(prefix + 'consequents.tsv', sep='\t', delimiter=None, header=None)
c_df = pd.DataFrame({
    'id': c_df[0],
    'labels':c_df[1],
    'text': c_df[2],
    'start':c_df[3],
    'end': c_df[4]
})
s_df = pd.read_csv(prefix + 'simplified.tsv', sep='\t', delimiter=None, header=None)
s_df = pd.DataFrame({
    'id': s_df[0],
    'labels':s_df[1],
    'text': s_df[2]
})

rc_df = pd.read_csv(prefix + 'consequent_results.csv', header=None)
rc_df = pd.DataFrame({
    'id': rc_df[0],
    'labels':rc_df[1]
})
ra_df = pd.read_csv(prefix + 'antecedent_results.csv', header=None)
ra_df = pd.DataFrame({
    'id': ra_df[0],
    'labels':ra_df[1]
})
rs_df = pd.read_csv(prefix + 'simplified_results.csv', header=None)
rs_df = pd.DataFrame({
    'id': rs_df[0],
    'labels':rs_df[1]
})

a_df.head(10)

In [0]:
def guess(id, original, s_df):
  antecedent =str()
  consequent = str()
  astart=-1
  aend = -1
  cstart=-1
  cend = -1

  srule = s_df.loc[s_df.id.str.contains(id)]
  if len(srule) == 0:
    print(id, 'not found')
    return antecedent, astart, aend, consequent, cstart, cend  

  slab = '0'
  min = 10 # minimal reuired score on pre-trained transformer
  print(id,' guessing...')
  for s in srule.index: # s = index
    sid = srule['id'][s]
    stext = srule['text'][s]
    score = get_score(stext)
    if score < min :
      min = score
      if '|' in stext:
        texts = stext.split('|')
        if len(texts) > 0:
          antecedent = texts[0]
          if len(consequent) == 0 and len(texts) > 1 :
            consequent = texts[1]
                    
  if len(antecedent) == 0 and '|' in text:
    antecedent =  text.split('|')[0]

  if len(consequent) == 0 and '|' in text and len(text.split('|')) > 1:
    consequent =  text.split('|')[1]

  print(id, antecedent,' | ', consequent)
  
  if len(antecedent) > 0:
    if astart ==-1 or aend ==-1:
      try:
        astart = original.index(antecedent.rstrip())
        if aend > astart:
          antecedent = original[astart-1:aend]
          print(id,' antecedent: '+antecedent)
      except ValueError:
        astart = 0
        aend = 0
        print(id, ValueError)
        print(antecedent,'not in:\n', original)
    if aend ==-1:  
      aend = astart + len(antecedent) 
  else:
    print(id,' no antecedent')      
        
  if len(consequent) > 0 :
    if cstart ==-1 or cend ==-1:
      try:
        cstart = original.index(consequent.rstrip())
        if cend > cstart:
          consequent = original[cstart-1,cend]
          print(id,' consequent: '+consequent)  
      except ValueError:
        cstart = 0
        cend = 0
        print(id, ValueError)
        print(consequent,'not in:\n', original)
    if cend ==-1:   
      cend = cstart + len(consequent)
  else:
    print(id,' no consequent')

  return antecedent, astart, aend, consequent, cstart, cend

In [0]:
def analyze(id, df, r_df):
  atext = str()
  astart = -1
  aend = -1
  arule = df.loc[df.id.str.contains(id)]
  if len(arule) == 0:
    print(id, ' missing')
    return atext, astart, aend 

  # First check based on rules
  alab ='0' # assume antecedent false
  for a in arule.index: # [a] = index
    amid = arule['id'][a]  
    alab = arule['labels'][a] # rule
    astart = arule['start'][a]
    aend = arule['end'][a]
    atext = arule['text'][a]
    text = atext
    #print(amid,'candidate ='+atext)
    if alab == 1:
      return atext, astart, aend

    # If rules gave no certain "yes"(==1 above)
    apred = r_df.loc[r_df.id.str.contains(amid)]
    if len(apred) == 0:
      print(amid, ' not predicted')
      continue
    for p in apred.index: # [p] = index
      amid = apred['id'][p]  
      alab = apred['labels'][p] # prediction
      apredrule = df.loc[df.id.str.contains(amid)]
      if len(apredrule) == 0:
        print(amid, ' not found ')
        continue
      for i in apredrule.index:    
        astart = apredrule['start'][i]
        aend = apredrule['end'][i]
        atext = apredrule['text'][i]
        text = atext
        #print(amid,'tried ='+atext)
        if alab == 1: # if no prediction, pick last!
          return atext, astart, aend
  return atext, astart, aend

In [0]:
import pandas as pd

prefix = '/content/'

from tqdm import tqdm
from time import sleep

submit_df= pd.DataFrame(columns=['sentenceID','antecedent_startid','antecedent_endid','consequent_startid','consequent_endid'])

for ind in tqdm(test_df.index): 
  original = test_df['text'][ind]
  id = test_df['id'][ind]
  print(id)

  astart=-1
  aend = -1
  cstart=-1
  cend = -1
  text= str()
  
  antecedent, astart, aend = analyze(id, a_df, ra_df)
  consequent, cstart, cend = analyze(id, c_df, rc_df)

  if len(antecedent) > 0:
    text = antecedent + ' | ' + consequent
  else:
    text = ' | ' + consequent

  if len(text) == 0: # no antecedent or consequent
    if len(srule) == 0:
      print(id, 'simplified sentence is not available!')
      continue
    antecedent, astart, aend, consequent, cstart, cend = guess(id,original,s_df)  

  if len(antecedent) > 0:
    text = antecedent + ' | ' + consequent
  else:
    text = ' | ' + consequent


  result_row = pd.Series( data={
    'sentenceID': id,
    'antecedent_startid':astart,
    'antecedent_endid': aend,
    'consequent_startid': cstart,
    'consequent_endid': cend
  }, name = str(id) )

  submit_df = submit_df.append(result_row, ignore_index=True)

  print(id, text, astart, aend, cstart, cend)
  sleep(1)

submit_df.to_csv(prefix+'subtask2.csv',index=False, header=['sentenceID','antecedent_startid','antecedent_endid','consequent_startid','consequent_endid']) 
submit_df.head() 

In [0]:
!zip submission_task2.zip subtask2.csv
!mv /content/submission_task2.zip /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/